In [1]:
## Importing needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [ ]:
## Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.random.seed(42)
## Building class named Neural_Network
class Neural_Network:
  def __init__(self,Input_layer_size,Output_layer_size,Hidden_layer_size):
    self.Input_layer_size=Input_layer_size
    self.Hidden_layer_size=Hidden_layer_size
    self.Output_layer_size=Output_layer_size

    ## Weights and Bias Intialization
    self.Weights_I_H=np.random.randn(self.Input_layer_size,self.Hidden_layer_size) #Initializing weights for input to hidden layer of network
    self.Bias_H=np.zeros((1,self.Hidden_layer_size)) #Initializing bias to hidden layer
    self.Weights_H_O=np.random.randn(self.Hidden_layer_size,self.Output_layer_size) #Initializing weights for hidden to output layer
    self.Bias_O=np.zeros((1,self.Output_layer_size)) # Initializing bias to output layer
    self.DeltaWeight_I_H=np.zeros((self.Input_layer_size,self.Hidden_layer_size)) #Delta weights
    self.DeltaWeights_H_O=np.zeros((self.Hidden_layer_size,self.Output_layer_size)) #Delta weights

  ## defining Sigmoid activation function
  def sigmoid(self,x,lmda=0.1):
    return 1 / (1 + np.exp(-lmda * x))

  ## defining derivative of activation function
  def sigmoid_derivative(self,x,lmda=0.1):
    return ((lmda * x) * (1 - x))

  ## defining normalization function
  def Normalize_data(self,data):
    Min_value=np.min(data,axis=0)
    Max_value=np.max(data,axis=0)
    return (data-Min_value)/(Max_value-Min_value)

  ## defining function for spliting data
  def data_split(self,data,train_ratio=0.7, Val_ratio=0.15):

    Train_size=int(len(data) * train_ratio)
    Validation_size=int(len(data) * Val_ratio)

    Train_data = data[:Train_size]
    Validation_data = data[Train_size:(Train_size + Validation_size)]
    Test_data = data[(Train_size+Validation_size):]
    return Train_data , Validation_data , Test_data

  def prediction(self,x):
    hidden_layer_output=self.sigmoid(np.dot(x,self.Weights_I_H)+self.Bias_H)
    predicted_output=self.sigmoid(np.dot(hidden_layer_output,self.Weights_H_O)+self.Bias_O)
    return predicted_output

  # Feed Forward
  def Feed_Forward(self,x):
    self.Hidden=self.sigmoid(np.dot(x,self.Weights_I_H)+self.Bias_H)
    self.Output=self.sigmoid(np.dot(self.Hidden,self.Weights_H_O)+self.Bias_O)
    return self.Output

  # Back Propagation
  def Back_Propagation(self, x, y, learning_rate, momentum):
    ## Calculating Output Error
    Output_Error = y - self.Output
    Delta_Output_layer = Output_Error * self.sigmoid_derivative(self.Output)

    ## Calculating Error at Hidden
    Hidden_Error = Delta_Output_layer.dot(self.Weights_H_O.T)
    Delta_Hidden_layer = Hidden_Error * self.sigmoid_derivative(self.Hidden)

    ## Updating Weights and Bias
    self.Weights_H_O += (self.Hidden.T.dot(Delta_Output_layer) * learning_rate) + (momentum * self.DeltaWeights_H_O)
    self.DeltaWeights_H_O = (self.Hidden.T.dot(Delta_Output_layer) * learning_rate) + (momentum * self.DeltaWeights_H_O)
    self.Bias_O += np.sum(Delta_Output_layer, axis=0) * learning_rate
    self.Weights_I_H += (x.T.dot(Delta_Hidden_layer) * learning_rate) + (momentum * self.DeltaWeight_I_H)
    self.DeltaWeight_I_H = (x.T.dot(Delta_Hidden_layer) * learning_rate) + (momentum * self.DeltaWeight_I_H)
    self.Bias_H += np.sum(Delta_Hidden_layer, axis=0) * learning_rate

  def Training(self, x_Train, y_Train, x_Validation, y_Validation, x_Test, y_Test, learning_rate, momentum, epochs, early_stop_patience=5):
    Train_Losses = []
    Validation_Losses = []
    Test_Losses = []

    Best_Validation_Loss = float('inf')
    epochs_without_imp = 0

    for epoch in range(epochs):
        epoch_train_loss = 0

        for i in range(len(x_Train)):
            x_row = x_Train[i].reshape(1, -1)
            y_row = y_Train[i].reshape(1, -1)

            self.Feed_Forward(x_row)
            self.Back_Propagation(x_row, y_row, learning_rate, momentum)

            # Calculate and store training loss for the current row
            epoch_train_loss += np.sqrt(np.mean(np.square(y_row - self.Feed_Forward(x_row))))

        # Calculate average training loss for the epoch
        avg_train_loss = epoch_train_loss / len(x_Train)
        Train_Losses.append(avg_train_loss)

        # Calculate and store validation loss for the entire validation set
        validation_loss = 0
        for i in range(len(x_Validation)):
            x_row_validation = x_Validation[i].reshape(1, -1)
            y_row_validation = y_Validation[i].reshape(1, -1)
            validation_loss += np.sqrt(np.mean(np.square(y_row_validation - self.Feed_Forward(x_row_validation))))

        avg_validation_loss = validation_loss / len(x_Validation)
        Validation_Losses.append(avg_validation_loss)

        # Calculate and store test loss for the entire test set
        test_loss = 0
        for i in range(len(x_Test)):
            x_row_Test = x_Test[i].reshape(1, -1)
            y_row_Test = y_Test[i].reshape(1, -1)
            test_loss += np.sqrt(np.mean(np.square(y_row_Test - self.Feed_Forward(x_row_Test))))

        avg_test_loss = test_loss / len(x_Test)
        Test_Losses.append(avg_test_loss)

        print("\nEpoch:", epoch, "Training Loss:", avg_train_loss, "Validation Loss:", avg_validation_loss, "Test Loss:", avg_test_loss)

        if avg_validation_loss < Best_Validation_Loss:
            Best_Validation_Loss = avg_validation_loss
            epochs_without_imp = 0
        else:
            epochs_without_imp += 1

        if epochs_without_imp >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} owing to no improvement for {early_stop_patience} epochs.')
            break

    self.plot_losses(Train_Losses, Validation_Losses)
    return Train_Losses, Validation_Losses, Test_Losses


  def plot_losses(self,Train_Losses,Validation_Losses):
    epochs=range(0 , len(Train_Losses))
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, Train_Losses, label='Training Loss')
    plt.plot(epochs, Validation_Losses, label='Validation Loss')
    plt.title('Training and Validation Losses Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error Loss')
    plt.legend()
    plt.show()
## Load Data

data=pd.read_csv('/content/drive/MyDrive/ce889_dataCollection.csv')

NN=Neural_Network(Input_layer_size = 2 , Hidden_layer_size =11, Output_layer_size = 2 )

Normalized_Data = NN.Normalize_data(data.values)

# Split data into train, validation, and test sets
Normalized_Data_Shuffled = shuffle(Normalized_Data, random_state=42)
Train_data, Validation_data, Test_data = NN.data_split(Normalized_Data_Shuffled)

# Separate features and label
x_Train, y_Train = Train_data[:, :2], Train_data[:, :2]
x_Validation, y_Validation = Validation_data[:, :2], Validation_data[:, :2]
x_Test, y_Test = Test_data[:, :2], Test_data[:, :2]

## Train Neural Network
Train_Losses, Validation_Losses, Test_Losses = NN.Training(x_Train, y_Train, x_Validation, y_Validation, x_Test, y_Test, learning_rate=0.1, momentum=0.8, epochs=100)

prediction = NN.prediction(x_Test)
print("Predicted Output (Velocity X, Velocity Y):", prediction)

In [ ]:
NN.Weights_I_H

In [ ]:
NN.Weights_H_O

In [ ]:
NN.Bias_H

In [ ]:
NN.Bias_O